In [2]:
import os
from ultralytics import YOLO
import zipfile
import cv2

# 작업 경로 설정 (로컬 경로)
base_path = "/Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8"
os.makedirs(base_path, exist_ok=True)


In [ ]:
# ultralytics 설치 (터미널에서 실행 필요)
# pip install ultralytics

# YOLO 설치 확인
# YOLO.checks()


In [ ]:
# 데이터셋 다운로드 및 압축 해제
dataset_url = "https://public.roboflow.com/ds/EUBO4ATjbt?key=dnYrjDbMEB"
zip_path = os.path.join(base_path, "Aquarium_DataSet.zip")
extract_path = os.path.join(base_path, "Aquarium_DataSet")

# wget 대신 urllib 사용
import urllib.request
urllib.request.urlretrieve(dataset_url, zip_path)

if not os.path.exists(extract_path):
    os.makedirs(extract_path)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Aquarium_DataSet 압축 해제가 완료되었습니다. 폴더 경로: {extract_path}")


In [ ]:
# YAML 파일 생성
yaml_path = os.path.join(extract_path, "Aquarium_Data.yaml")
data = {
    'train': os.path.join(extract_path, "train", "images") + "/",
    'val': os.path.join(extract_path, "valid", "images") + "/",
    'test': os.path.join(extract_path, "test", "images") + "/",
    'nc': 7,
    'names': ['fish', 'jellyfish', 'penguin', 'puffin', 'shark', 'starfish', 'stingray']
}

import yaml
with open(yaml_path, 'w') as f:
    yaml.dump(data, f)

print("YAML 파일 생성 완료:", yaml_path)


In [ ]:
# 사전 학습된 YOLO 모델 로드
model = YOLO("yolov8n.pt")  # 사전 학습된 모델 로드

# 모델 학습
results = model.train(
    data=yaml_path,         # 데이터셋 구성 파일 경로
    epochs=50,             # 학습 에포크 수
    patience=30,            # 조기 종료 기준 에포크 수
    batch=64,               # 배치 크기
    imgsz=416,              # 입력 이미지 크기
    project=base_path,      # 결과 저장 경로
    name='modelresult',     # 프로젝트 이름
    device="mps"            # Apple Silicon GPU(MPS) 사용
)

print(f"모델 학습 완료. 저장 경로: {results.save_dir}")


In [3]:
# 예측 실행 (이미지)
model = YOLO(os.path.join(base_path, "modelresult", "best.pt"))

test_image = os.path.join(extract_path, "test", "images", "IMG_2301_jpeg_jpg.rf.2c19ae5efbd1f8611b5578125f001695.jpg")
output_dir = os.path.join(base_path, "modelresultImg/")
os.makedirs(output_dir, exist_ok=True)

results = model.predict(
    source=test_image,
    save=True,
    save_dir=output_dir
)

print(f"결과 이미지가 다음 경로에 저장되었습니다: {output_dir}")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/modelresult/best.pt'

In [ ]:
# 비디오 예측 실행
video_path = "/Users/your_username/Documents/YOLOTest7/sample_video.mp4"
output_video = os.path.join(base_path, "output_video.mp4")

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Cannot open video file.")
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, save=False, conf=0.5)
    annotated_frame = results[0].plot()
    out.write(annotated_frame)

cap.release()
out.release()
print(f"Detection completed. Output video saved at {output_video}")
